In [5]:
"""
The goal of assign_multiple_gt_per_box is to prove we can overfit to a single selected proposal.  Reasoning:
- With the customized head, we need to specialize the loss with respect to each proposal
- With the customized head, we may be using only a subset of the proposals, or a different set entirely (interfacing
at an earlier level).  By fitting to a single proposal, we're demonstrating that we are able to interface at the
proposal rather than image level.
- Our next step will be to change this loss to the customized loss.
"""

import torch
import torch.distributed

from detectron2.data import MetadataCatalog
import vis_utils
from detectron2.utils.events import EventStorage
import script_utils
from trainer_apd import Trainer_APD

exporter_ = None


def dbprint(*args, **kwargs):
    print(*args, **kwargs)


def train_on_single_image(trainer: Trainer_APD, datapoint, max_itr=100, start_itr=0):
    assert max_itr >= start_itr
    with EventStorage(0) as trainer.storage:
        for t in range(max_itr - start_itr):
            trainer.run_step_with_given_data(datapoint)

            
image_id = '486536'

exporter = vis_utils.FigExporter()

print('Beginning setup...')
cfg = script_utils.get_custom_maskrcnn_cfg()
trainer = Trainer_APD(cfg)

print('Completing setup...')
standard_head = trainer.model.roi_heads.mask_heads['standard']
custom_head = trainer.model.roi_heads.mask_heads['custom']

datapoint = torch.load(script_utils.get_datapoint_file(cfg, image_id))

if type(datapoint) is list:
    datapoint = datapoint[0]
batch = [datapoint]

# head_type = 'custom' if custom_head else 'standard'
head_type = 'custom'
script_utils.activate_head_type(trainer, head_type)
cfg_tag = f"_{head_type}"

metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
names = metadata.get("thing_classes", None)
labels = [names[i] for i in datapoint['instances'].gt_classes]


Config './detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x_APD.yaml' has no VERSION. Assuming it to be compatible with latest v2.


Beginning setup...
[Building model]
Elapsed: 0.5848042964935303
[Building dataloader]
Elapsed: 30.87593722343445
Completing setup...
